In [1]:
import os
import pandas as pd
from pandas import (
    merge,
    read_csv,
    DataFrame,
    to_datetime,
    to_numeric,
)
from os.path import join
from bson.tz_util import FixedOffset
import numpy as np
import pymysql
from math import acos, sin, cos, atan2, sqrt
import os, sys
import datetime
from datetime import datetime, timedelta
import ast
import warnings
warnings.filterwarnings("ignore")


from s3_handler import S3Handler
from get_footprint_data_from_blob import FootPrintData
from database_manager import DatabaseManager
from mongo_manager import MongoDBManager
from base64converter import base64_converter
from email_sender import use_email
from distance_calculation_helper import (
    to_radian,
    get_haversine_distance,
    tag_patterned,
    delivery_drop_distance,
    pickup_restaurant_distance,
    pickedUp_time,
    mysql_rides_to_dataframe,
    json_concat_and_conversion,
    remove_footprint_data_directory
    
)
from weekly_provider_profiling_3_0 import (
    weekly_provider_profiling
)

In [2]:
#download_data_from_blob
footprint_data = FootPrintData()
footprint_data.get_data(start=2,end=2)
                                 

11-07-2021


In [3]:
def food_orders_extract(number):
    
    date = datetime.strftime(datetime.now() - timedelta(number), '%Y-%m-%d')
    print(date)
    databasemanager = DatabaseManager(file='db_creds_live_foods.json')
    sql = "select columns names from database_name.table_name   where date(created_at)='"+date+"'  "

    databasemanager.execute(sql = sql)
    df_food = pd.DataFrame(databasemanager.result_)
    print(df_food.shape)
    
    return df_food
    

In [4]:
def food_orders_calculations(df):
    
    df['payment_difference_between_user_restaurant'] = df['payable_by_user'] - df['restaurant_payable']
    df['pickedup_at'] = df['pickedup_at'].apply(lambda x: pd.to_datetime(x,errors = 'coerce'))
    df['accepted_at'] = df['accepted_at'].apply(lambda x: pd.to_datetime(x,errors = 'coerce'))
    df['delivered_at'] = df['delivered_at'].apply(lambda x: pd.to_datetime(x,errors = 'coerce'))
    df['pickedup_at'] = df['pickedup_at'].apply(lambda x: pd.to_datetime(x,errors = 'coerce'))
    df["time_accepted_at_to_pickedup_at"] = (df["pickedup_at"] - df["accepted_at"])
    df["time_pickedup_at_to_delivered_at"] = (df["delivered_at"] - df["pickedup_at"])
    df['dist_accepted_pickedup'] = 0
    df['dist_pickedup_delivered'] = 0
    df["count_coordinates_accepted_pickedup"] =  df["count_coordinates_pickedup_delivered"] = 0
    df["coordinates_accepted_pickedup"] = 0
    df["coordinates_pickedup_delivered"]  = 0
    
    return df


In [5]:
def coordinate_assigment(df_food,coordinates):
    for i in range(len(df_food)):
        dummydf_ap = coordinates[(coordinates.providerid==df_food.loc[i, 'provider_id']) & 
                                (coordinates.datetime_formatted >=df_food.loc[i, 'accepted_at']) & 
                                (coordinates.datetime_formatted <=df_food.loc[i, 'pickedup_at'])]
        dummydf_ap.reset_index(inplace = True)
        #print(dummydf_ap)
        distance_ap=0
        list_coords= []
        for j in range(1,len(dummydf_ap)):

            coord1 = (dummydf_ap['lat'][j-1], dummydf_ap['lng'][j-1])
            coord2 = (dummydf_ap['lat'][j], dummydf_ap['lng'][j])
            list_coords.append(coord1)
            dist = get_haversine_distance(coord1, coord2)
            distance_ap+=dist

        df_food['dist_accepted_pickedup'][i] = distance_ap
        df_food["count_coordinates_accepted_pickedup"][i] = len(list_coords)
        df_food["coordinates_accepted_pickedup"][i] = str(list_coords)

        dummydf_pd = coordinates[(coordinates.providerid==df_food.loc[i, 'provider_id']) & 
                                (coordinates.datetime_formatted >=df_food.loc[i, 'pickedup_at']) & 
                                (coordinates.datetime_formatted <=df_food.loc[i, 'delivered_at'])]
        dummydf_pd.reset_index(inplace = True)
        #print(dummydf_pd)
        distance_pd=0
        list_coords_dd= []
        for j in range(1,len(dummydf_pd)):

            coord1 = (dummydf_pd['lat'][j-1], dummydf_pd['lng'][j-1])
            coord2 = (dummydf_pd['lat'][j], dummydf_pd['lng'][j])        
            list_coords_dd.append(coord1)
            dist = get_haversine_distance(coord1, coord2)
            distance_pd+=dist

        df_food['dist_pickedup_delivered'][i] = distance_pd
        df_food["count_coordinates_pickedup_delivered"][i] = len(list_coords_dd)
        df_food["coordinates_pickedup_delivered"][i] = str(list_coords_dd)
        
    print('Done coordinate assignment')
        
    return df_food   
    

In [6]:
def threshold_tag(df_food):
    df_food['coord_pattern']=0
    df_food['coord_pattern'] = df_food.apply(lambda row: tag_patterned(row['coordinates_accepted_pickedup'], row['coordinates_pickedup_delivered']), axis=1)
    ##
    print('Pattern tagging')
    ##
    df_food['lt100m'] = df_food['dist_pickedup_delivered'].apply(lambda x: 1 if x <100 else 0)
    df_food['lt50m'] = df_food['dist_pickedup_delivered'].apply(lambda x: 1 if x <50 else 0)
    df_food['gt10km'] = df_food['dist_pickedup_delivered'].apply(lambda x: 1 if x >10000 else 0)
    df_food['gt20km'] = df_food['dist_pickedup_delivered'].apply(lambda x: 1 if x >20000 else 0)
    
    return df_food

In [9]:
number=1

df_food = food_orders_extract(number)
print(df_food.shape)

df_food = food_orders_calculations(df_food)
print('df_food_updated')
print(df_food.columns)
print(df_food.shape)

date = datetime.strftime(datetime.now() - timedelta(number), '%d-%m-%Y')
coordinates = json_concat_and_conversion(date)
print(coordinates.columns)
print(coordinates.shape)


df_food = coordinate_assigment(df_food=df_food,coordinates=coordinates)
##
print('Done coordinate assignment')
##

df_food = threshold_tag(df_food=df_food)

df_food['time_accepted_at_to_pickedup_at'] = pd.to_timedelta(df_food.time_accepted_at_to_pickedup_at)
df_food['time_pickedup_at_to_delivered_at'] = pd.to_timedelta(df_food.time_pickedup_at_to_delivered_at)

df_food['time_accepted_at_to_pickedup_at'] = df_food['time_accepted_at_to_pickedup_at'].apply(lambda x: x.seconds/60)
df_food['time_pickedup_at_to_delivered_at'] = df_food['time_pickedup_at_to_delivered_at'].apply(lambda x: x.seconds/60)

filename = date+'_coordinate_foods.csv'
df_food.to_csv(filename, index = False)
#s3handler.upload_single_file(filename, 'coordinates/foods/'+date+'_coordinate_foods.csv')
# os.remove(filename)
print('File Exported')
    
# In[23]:

2021-07-12
(1504, 17)
(1504, 17)
df_food_updated
Index(['created_at', 'order_id', 'user_id', 'provider_id', 'restaurant_name',
       'restaurant_lat', 'restaurant_lng', 'restaurant_payment_type',
       'delivery_lat', 'delivery_lng', 'accepted_at', 'pickedup_at',
       'delivered_at', 'res_has_panel', 'payable_by_user',
       'restaurant_payable', 'coupon_code',
       'payment_difference_between_user_restaurant',
       'time_accepted_at_to_pickedup_at', 'time_pickedup_at_to_delivered_at',
       'dist_accepted_pickedup', 'dist_pickedup_delivered',
       'count_coordinates_accepted_pickedup',
       'count_coordinates_pickedup_delivered', 'coordinates_accepted_pickedup',
       'coordinates_pickedup_delivered'],
      dtype='object')
(1504, 26)
json append
json converted
Index(['lat', 'lng', 'providerid', 'servicetypeid', 'TenantId', 'VerticalId',
       'datetime', 'datetime_formatted'],
      dtype='object')
(1980531, 8)
Done coordinate assignment
Done coordinate assignment
Pat

In [8]:
# weekly_provider_profiling()
# print('weekly provider profiling done')

    